In [15]:
import pandas as pd
import re

In [16]:
acidentes_2016_df = pd.read_csv("datatran2016.csv", delimiter=";", encoding='ISO-8859-1', decimal=",")
radares_df = pd.read_csv("dados_dos_radares.csv", delimiter=";", encoding='ISO-8859-1', decimal=",")
acidentes_2016_df = acidentes_2016_df[acidentes_2016_df["uf"].isin(["SC", "PR", "RS"])]
#acidentes_2016_df.km = acidentes_2016_df.km.map(lambda x: x.replace(",", ".")).astype(float)
radares_df = radares_df[radares_df['uf'].isin(["SC", "PR", "RS"])]
radares_df["rodovia"] = radares_df["rodovia"].map(lambda x: re.search(r'BR-(\d+)', x).group(1)).astype(int)
radares_df = radares_df.rename(columns={"km_m":"km"})
#radares_df.km = radares_df.km.map(lambda x: x.replace(",", ".")).astype(float)

# 1 - Selecionar a localização dos radares instalados após 2016

In [17]:
radares_df = radares_df[(radares_df["ano_do_pnv_snv"] <= 2016)]
radares_df.head()

,concessionaria,ano_do_pnv_snv,tipo_de_radar,rodovia,uf,km,municipio,tipo_pista,sentido,situacao,data_da_inativacao,latitude,longitude,velocidade_leve,velocidade_pesado
32,AUTOPISTA LITORAL SUL,2009,Redutor,376,PR,650.800,Tijucas do Sul,Principal,Crescente,Ativo,NaN,-25.8451,-49.0726,60,60
33,AUTOPISTA LITORAL SUL,2009,Redutor,376,PR,664.773,Guaratuba,Principal,Crescente,Ativo,NaN,-25.8582,-48.9512,60,60
34,AUTOPISTA LITORAL SUL,2009,Redutor,376,PR,667.013,Guaratuba,Principal,Crescente,Ativo,NaN,-25.8637,-48.9348,60,60
35,AUTOPISTA LITORAL SUL,2009,Redutor,376,PR,668.334,Guaratuba,Principal,Crescente,Ativo,NaN,-25.8693,-48.9341,60,60
36,AUTOPISTA LITORAL SUL,2009,Redutor,376,PR,668.880,Guaratuba,Principal,Decrescente,Ativo,NaN,-25.8749,-48.9363,60,60


# 2 - Definir um raio de influencia dos radares

In [18]:
R = 1.5 # km (1 antes e 0.5 depois do radar) 
# TODO Definir algum criterio melhor, talvez com base nos acidentes perto dos radares ja instalados


# 3 - Agrupar os acidentes antes e depois da instalação dentro da área de influência

In [36]:

# Get radares_por_br
radares_por_brs = {per_br_df: group_df for per_br_df, group_df in radares_df.groupby("rodovia")}

# Ge acidentes_por_br
acidentes_por_brs = {per_br_df: group_df for per_br_df, group_df in acidentes_2016_df.groupby("br")}

# Encontrar total de acidentes/feridos na area de influencia de cada radar
acidentes_por_radar = pd.DataFrame(columns=["br", "km", "n_acidentes", "n_feridos_graves", "n_feridos_leves", "n_mortos"])
for br, radar_por_br in radares_por_brs.items():

    acidentes_br = acidentes_por_brs[br]

    for radar in radar_por_br.itertuples():
        acidentes_sum = 0
        graves, leves, mortos = 0, 0 , 0
        
        for acidente in acidentes_br.itertuples():
            if abs(acidente.km - radar.km) < R: 
                acidentes_sum+=1
                mortos+=acidente.mortos
                leves+=acidente.feridos_leves
                graves+=acidente.feridos_graves
                
        new_row = {
                'br':br,
                'km': radar.km,
                "municipio": radar.municipio,
                "uf": radar.uf,
                "n_acidentes": acidentes_sum,
                "n_feridos_graves" : graves,
                "n_feridos_leves" : leves,
                "n_mortos": mortos
            }
        acidentes_por_radar = acidentes_por_radar.append(new_row, ignore_index=True)
        

df = acidentes_por_radar.sort_values('km')

df['km_group'] = df.km.diff().gt(5).cumsum() + 1

# Nao tinha dados de radar na 116 antes de 2020
"""
to_save = pd.DataFrame(acidentes_por_radar).astype(str)
to_save.km = to_save.km.map(lambda x: x.replace(".", ","))
to_save.n_acidentes = to_save.n_acidentes.map(lambda x: x.replace(".", ","))
to_save.n_mortos = to_save.n_mortos.map(lambda x: x.replace(".", ","))
to_save.n_feridos_graves = to_save.n_feridos_graves.map(lambda x: x.replace(".", ","))
to_save.n_feridos_leves = to_save.n_feridos_leves.map(lambda x: x.replace(".", ","))
to_save.br = to_save.br.map(lambda x: x.replace(".", ",")).astype(str)
to_save
to_save.to_csv("acidentes_por_radar.csv")
""" 



C:\Users\rafae\AppData\Local\Temp\ipykernel_33620\2372794506.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  acidentes_por_radar = acidentes_por_radar.append(new_row, ignore_index=True)
C:\Users\rafae\AppData\Local\Temp\ipykernel_33620\2372794506.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  acidentes_por_radar = acidentes_por_radar.append(new_row, ignore_index=True)
C:\Users\rafae\AppData\Local\Temp\ipykernel_33620\2372794506.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  acidentes_por_radar = acidentes_por_radar.append(new_row, ignore_index=True)
C:\Users\rafae\AppData\Local\Temp\ipykernel_33620\2372794506.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fu

br                                                               4203
km                                                            7440.18
n_acidentes                                                      3926
n_feridos_graves                                                  611
n_feridos_leves                                                  2770
n_mortos                                                           71
municipio           JoinvilleJoinvilleItajaíItajaíItajaíBalneário ...
uf                  SCSCSCSCSCSCSCSCSCSCSCSCSCSCSCSCSCSCSCSCSCSCSC...
km_group                                                          127
dtype: object